# 테스트 6회 할 차례

In [5]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(
            start_page)
        print("생성url: ", url)
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)


def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url


#####뉴스크롤링 시작#####

# 테스트 횟수
num = int(input('몇 번째 테스트? '))
# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")
# 검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []

# 일단 제목 수준으로 진행 (기사 내용은 추후에 적용 여부 판단)
# news_contents = []

news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
# ------------------------------------------------------------------------------------------------------------------
    # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
#     content = news_html.select("div#dic_area")
#     if content == []:
#         content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    #content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
#     content = re.sub(pattern=pattern1, repl='', string=content)
#     pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
#     content = content.replace(pattern2, '')
    
    news_titles.append(title)
#    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", (page2 + 1 - page) * 10, '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
#print("\n[뉴스 내용]")
#print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
#print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles})
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv(f'csv/{search} 뉴스원문{num}.csv', encoding='utf-8-sig', index=False)

print(type(news_df['title']))
# print(type(news_df['content']))


몇 번째 테스트? 1
검색할 키워드를 입력해주세요:카카오

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):50

크롤링할 종료 페이지:  50 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=카카오&pd=4&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&

100%|████████████████████████████████████████████████████████████████████████████████| 318/318 [01:15<00:00,  4.19it/s]

검색된 기사 갯수: 총  500 개

[뉴스 제목]
["카카오, 세 번째 ESG보고서…'서비스 장애' 대응 소개", '카카오페이증권, 미국 주식 환율우대 신청 서비스 오픈', '카카오, 2022 ESG 보고서 발간…지배구조 선진화 나선다', '카카오페이 “대출갈아타기 사전신청자 70% 1금융권 고객”', '패션바이카카오, ‘헬로 썸머 위크’ 진행', '카카오TV의 씁쓸한 퇴장…위기의 국내 OTT', '네이버 수준으로 설비투자 늘리는 카카오… ‘먹통 카톡’ 오명 벗을까', '카카오, 문어발식 확장 멈춘다더니… 계열사 더 늘었다', 'SPC·카카오·매일유업…사옥을 짓지 않는 이유는', '관광公·카카오·티맵 등 외래객 교통편의 공조 활발', '카카오 이어 악사손보… ‘교보지주 꿈’ 이룰까', '크리스피크림도넛, 카카오톡 선물하기 전용 굿즈 상품 출시', '카카오페이증권, 미 주식 협의수수료 신설…월 5억 이상 거래 조건', '카카오 등 IT업체도 알뜰폰 사업 속도···통신비 내려갈까', '[기자의 눈] 광고창 된 카카오톡', "'멋지다 박연진'…배우 임지연, 명예경찰 됐다", '尹대통령, 한상혁 방통위원장 면직안 재가 초읽기', '31일부터 신용대출·카드론 갈아탄다', '“당장 안하면 호갱됩니다” 15분 클릭, 대출이자 수십만원 아끼세요', "'돈봉투 체포동의안'에 민주당, 간호법 재의결로 '입법 반격'", '더 낮은 금리 신용대출 발견하면? 15분 만에 갈아탄다', "비공개 선고 읍소까지…'강제추행' 前아이돌 1심 징역형", '"내일부터 금리 경쟁 불붙는다"…\'대환대출플랫폼\' 31일 본격 출시', '방한관광객 이동 편리해진다…관광공사-교통업계 맞손', '‘앗! 여기가 더 낮네’ 휴대전화로 내일부터 신용대출 갈아탈 수 있다', "'초격차 기술로 불황 넘자'…삼성·SK, 차세대 D램 잇달아 양산", "경찰, '한동훈 개인정보 유출 의혹'…MBC 사옥 압수수색 시도(종합)", '경찰, 거액 수임료 의혹 민주당 양부남 법률위원장 영장 신청', '\'盧부부 명예훼손\'